In [2]:
pip install pyodbc pandas

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pyodbc
import pandas as pd

In [4]:
conn = pyodbc.connect(
    'DRIVER={SQL Server};'
    'SERVER=LAPTOP-67LVVL1C;'
    'DATABASE=BoardGameDB1;'
    'Trusted_Connection=yes;'
)

In [5]:
print(pyodbc.drivers())

['SQL Server', 'SQL Server Native Client RDA 11.0', 'ODBC Driver 17 for SQL Server', 'Microsoft Access Driver (*.mdb, *.accdb)', 'Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)', 'Microsoft Access Text Driver (*.txt, *.csv)', 'Microsoft Access dBASE Driver (*.dbf, *.ndx, *.mdx)']


In [6]:
cursor = conn.cursor()
cursor.execute("SELECT * FROM BoardGames")

In [7]:
rows = cursor.fetchall()
columns = [column[0] for column in cursor.description]
df = pd.DataFrame.from_records(rows, columns=columns)

In [8]:
df.head(5)

,row_id,boardgame,release_year,min_players,max_players,min_playtime,max_playtime,minimum_age,avg_rating,num_ratings,complexity,rank_overall,owned,wishlisted,total_plays,fans,page_views,description
0,1,Brass: Birmingham,2018,2,4,60,120,14,8.57,53548,3.87,1,76201,20334,152594,6726,5555379,"Build networks, grow industries, and navigate ..."
1,2,Pandemic Legacy: Season 1,2015,2,4,60,60,13,8.51,56071,2.83,2,87378,14478,301410,4221,4592660,Mutating diseases are spreading around the wor...
2,3,Ark Nova,2021,1,4,90,150,14,8.54,54528,3.79,3,79476,16121,528979,7465,6113759,"Plan and build a modern, scientifically manage..."
3,4,Gloomhaven,2017,1,4,60,120,14,8.56,65443,3.92,4,101913,21597,542336,10780,13606453,Vanquish monsters with strategic cardplay. Ful...
4,5,Twilight Imperium: Fourth Edition,2017,3,6,240,480,14,8.57,26686,4.34,5,32012,12449,49484,3546,4354588,"Build an intergalactic empire through trade, r..."


Which boardgame has the lowest avg_rating?

In [24]:
low_rating = df.groupby('boardgame')['avg_rating'].min().sort_values().head(1)
print('Lowest Rated Board Game is:', low_rating.index[0], 'with rating:', low_rating.values[0])

Lowest Rated Board Game is: Jungle Speed with rating: 6.46


What is the median number of total_plays across all games?

In [23]:
median_total_plays = df['total_plays'].median()
print('Median of total plays across all games is:', median_total_plays)

Median of total plays across all games is: 16004.0


Which boardgame has the highest complexity?

In [22]:
high_complex = df.groupby('boardgame')['complexity'].max().sort_values(ascending=False).head(1)
print('Most Complex Board Game is:', high_complex.index[0], 'with highest complexity:', high_complex.values[0])

Most Complex Board Game is: High Frontier 4 All with highest complexity: 4.82


Create new column RatingCategory ("High" if avg_rating ≥ 8 else "Low"). Show first 10 rows.

In [21]:
def categorize(rating):
    if rating >= 8:
        return 'High'
    else:
        return 'Low'

df['RatingCategory'] = df['avg_rating'].apply(categorize)
print(df[['boardgame','avg_rating','RatingCategory']].head(10))

                           boardgame  avg_rating RatingCategory
0                  Brass: Birmingham        8.57           High
1          Pandemic Legacy: Season 1        8.51           High
2                           Ark Nova        8.54           High
3                         Gloomhaven        8.56           High
4  Twilight Imperium: Fourth Edition        8.57           High
5                     Dune: Imperium        8.42           High
6                  Terraforming Mars        8.35           High
7    War of the Ring: Second Edition        8.55           High
8               Star Wars: Rebellion        8.42           High
9          Dune: Imperium – Uprising        8.71           High


Top 5 boardgames with the highest owned count

In [25]:
top5_owned = df.groupby('boardgame')['owned'].max().sort_values(ascending=False).head(5)
print(top5_owned)

boardgame
CATAN             226927
Pandemic          217531
Carcassonne       212595
7 Wonders Duel    174404
Codenames         170102
Name: owned, dtype: int64


Which release_year has the lowest average rating?

In [26]:
low_avg_rate_year = df.groupby('release_year')['avg_rating'].mean().sort_values().head(1)
print('Release year with lowest avg rating is:', low_avg_rate_year.index[0], 'with rating:', low_avg_rate_year.values[0])

Release year with lowest avg rating is: 1973 with rating: 6.63


Summary statistics (mean, min, max) of total_plays

In [ ]:
mean_total_plays = round(df['total_plays'].mean(),2)
min_total_plays = round(df['total_plays'].min(),2)
max_total_plays = round(df['total_plays'].max(),2)

print(f'Mean of total plays is: {mean_total_plays}, Minimum of total plays is: {min_total_plays}, Maximum of total plays is: {max_total_plays}')

From the DataFrame, select the first 10 boardgames with complexity greater than 4.0 and avg_rating below 7.

In [27]:
filtered_games = df[(df['complexity'] > 4.0) & (df['avg_rating'] < 7)].head(10)

print(filtered_games[['boardgame', 'complexity', 'avg_rating']])

Empty DataFrame
Columns: [boardgame, complexity, avg_rating]
Index: []


Series of average total_plays per release_year (top 5 years with highest values)

In [28]:
avg_plays = df.groupby('release_year')['total_plays'].mean().sort_values(ascending=False).head(5)
print(avg_plays)

release_year
1475    285209.0
1993    262246.0
1876    240174.0
1980    170778.0
1630    136165.0
Name: total_plays, dtype: float64


Average Complexity per Minimum Age Group

In [29]:
age_complexity = df.groupby('minimum_age')['complexity'].mean().sort_values()
print(age_complexity)

minimum_age
4     1.045000
5     1.192000
6     1.399474
7     1.577143
8     1.712783
9     2.144722
10    2.165810
11    2.578571
17    2.618333
13    2.820208
12    2.971145
14    2.971488
16    2.990714
18    3.330000
15    3.378571
Name: complexity, dtype: float64


Create a Series of average playtime (max_playtime – min_playtime) grouped by release_year using indexing and display the top 5 years with the highest values.


In [30]:
df['playtime_diff'] = df['max_playtime'] - df['min_playtime']
avg_playtime_by_year = df.groupby('release_year')['playtime_diff'].mean().sort_values(ascending=False).head(5)
print('Top 5 release years with highest average playtime difference are:\n', avg_playtime_by_year)

Top 5 release years with highest average playtime difference are:
 release_year
1983    285.0
1985    195.5
1979    100.0
1986     84.0
1980     70.0
Name: playtime_diff, dtype: float64


Which release_year has the maximum number of boardgames?

In [31]:
max_games_year = df['release_year'].value_counts().sort_values(ascending=False).head(1)
print('Release year with maximum number of boardgames is:', max_games_year.index[0], 'with', max_games_year.values[0], 'games')

Release year with maximum number of boardgames is: 2019 with 152 games


Top Categories by Engagement

In [9]:
df['engagement_score'] = df['total_plays'] + df['fans'] + df['owned']
top_engaged = df.sort_values('engagement_score', ascending=False).head(10)
print(top_engaged[['boardgame','engagement_score','avg_rating']])

                boardgame  engagement_score  avg_rating
139              Dominion            997563        7.60
89                   Azul            992332        7.73
34               Wingspan            946400        8.01
232           Carcassonne            928473        7.41
6       Terraforming Mars            903370        8.35
87    Race for the Galaxy            838526        7.74
166  Magic: The Gathering            813927        7.59
155             Codenames            806056        7.53
108             7 Wonders            798133        7.67
20         7 Wonders Duel            785386        8.08


Complexity Category & Average Metrics

In [13]:
def complexity_level(x):
    if x < 2.5: return 'Easy'
    elif x < 3.5: return 'Medium'
    elif x < 4.0: return 'Hard'
    else: return 'Expert'

df['ComplexityLevel'] = df['complexity'].apply(complexity_level)
summary = df.groupby('ComplexityLevel')[['avg_rating','owned','fans','wishlisted']].mean().round(2)
print(summary)

                 avg_rating     owned     fans  wishlisted
ComplexityLevel                                           
Easy                   7.25  16699.64   562.79     2128.57
Expert                 7.99  10884.07  1083.73     3372.31
Hard                   7.78  14508.90  1079.28     3261.22
Medium                 7.50  12350.78   751.92     2573.31
